Assignment 1 - Joanna Orzechowska

This code has 3 parts:

Admin tasks: import libraries and load the data and look at the data

Preprocessing: cleaning, imputing and normalizing data

Model: assessing the accuracy of the model - spoiler alert: 0.855

I will put the important information before the code (including the results)

AI disclosure: ChatGPT was used for troubleshooting, clarifying a few concepts (ex IQR), help with strings describing functions and readability of the readme file.

I did not however use AI to fix grammar and readability of this document. I'm sorry.

Step 1: Admin tasks
- importing all necessary libraries
- loading the dataset
- initial look into the data
I used Data Wrangler extension to visualize data. Below is the data wrangler summary:

Data shape      1,196 rows x 28 columns

Rows with missing values        1,148 (96.0%)

Duplicate rows      0 (0.0%)

In [ ]:
# Import necessary modules
import data_preprocessor as dp
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
messy_data = pd.read_csv('../Data/messy_data.csv', na_values=['', 'NA', 'Na', 'null', ""])
working_data = messy_data.copy()

# look into the messy data
print(messy_data.head())
print(messy_data.info())
print(messy_data.describe())
print(working_data.isnull().sum())

Step 2: Cleaning the data

This step is divided into multiple substeps, all results are saved into their own csv:

1. remove duplicates        -       cleaning_step1.csv
2. delete columns with >= 25% of missing values     -       cleaning_step2.csv
3. remove rows with >= 25% of missing values        -       cleaning_step3.csv
4. delete redundant features        -       cleaning_step4.csv
5. delete rows with missing value in the target column      -       cleaning_step5.csv
6. remove outliers      -       cleaning_step_outliers.csv (plot twist)
7. impute missing values        -       cleaning_step6.csv
8. normalize data       -       clean_data.csv

Some of the steps just neede to be there, no matter the order, others like removing outliers I wanted to do on the cleanest possible data before imputing and normalizing.

The cells need to be ran in an order, no skipping as they build on each other.

2.1 Removing duplicates. 
According to Data Wrangler, there were no duplicate rows. After I ran the function, the stats were the same as the messy data.

In [ ]:
# 2.1 remove duplicates

working_data = dp.remove_duplicates(working_data)
working_data.to_csv(r"../Data/cleaning_step1.csv", index=False)

2.2 Removing columns with >=25% missing data

The model needs data to predict data. There's no point keeping columns with 25+% of values missing.

Stats after:

Data shape 1,196 rows x 23 columns

Rows with missing values    752 (62.9%)

In [ ]:
# 2.2 remove columns with missing data

working_data = dp.remove_columns_missing_data(working_data)
working_data.to_csv(r"../Data/cleaning_step2.csv", index=False)

2.3 Removing rows with >=25% missing data

In retrospect, I should have probably done this step after deleting redundant features but instead I calculated the fraction of rows with missing 25+% of data to know how much data I'm deleting (6.18%). 

Stats after:

Data shape  1,122 rows x 23 columns

Rows with missing values    678 (60.4%)

In [ ]:
# 2.3.1 fraction of rows that have 25%+ of missing data

fraction = dp.count_missing_data_rows(working_data)
print(f"Fraction of rows with >=25% missing data fraction: {round(fraction,2)}")

# 2.3.2 remove rows with 25+% missing values
working_data = dp.remove_rows_missing_data(working_data)
working_data.to_csv(r"../Data/cleaning_step3.csv", index=False)

2.4 Deleting redundant features

First I looked into the data. 2 possible clusters became apparent: columns o,p and q, and columns r,u and y. I used to corr() function to compare only the columns I chose. They all showed correlation of 0.98 and above. I deleted all but r and p as those columns had the least % of missing values and most distinct values as per Data Wrangler.

Then I decided to compare all numerical columns left and saved the resulting data frame as "x.csv". I went rogue and used conditional formatting in excel to see if any correlation was above 0.9 and I discovered 2 other correlations (both above 0.98) and deleted colums c and n.

Stats after:

Data shape  1,122 rows x 17 columns

Rows with missing values    458 (40.8%)

In [ ]:
# 2.4.1 exploration into redundant features
print(working_data[["o","p","q"]].corr())
print(working_data[["r","u","y"]].corr())
# leaving r and p
x=working_data[["b","c","f","h","k","l","n","p","r","t","v","w"]].corr()
x.to_csv(r"../Data/x.csv", index=False)

# 4.2 deleting redundant features
working_data = working_data.drop(columns =["o","q","u","y","c","n"])
working_data.to_csv(r"../Data/cleaning_step4.csv", index=False)

2.5 Delete rows with missing target

I first checked how rows have a many missing target (0.23). And then I deleted them as imputing values would assign either 1 or 0, skewing learning.

Stats after:

Data shape     865 rows x 17 columns

Rows with missing values    201 (23.2%)

In [ ]:
# 5.1 fraction of rows with missing target
fraction = working_data["target"].isnull().mean()
print(f"Fraction of rows with missing target data: {round(fraction,2)}")

# 5.2 delete rows with missing target
working_data = working_data.dropna(subset = "target")
working_data.to_csv(r"../Data/cleaning_step5.csv", index=False)

2.6 Outliers

First I made box plots of each numerical column to see which columns have outliers. I used IQR method to deleted those rows. I wanted to do this step before imputing and especially normalizing so the outliers don't skew the mean/median/mode.

Stats after:

Data shape  690 rows x 17 columns

Rows with missing values    98 (14.2%)

In [ ]:
# 2.6.1 visualization
for column in ["b","f","k","l","p","r","v","w","h","t","target","a","i"]:
    sns.boxplot(x=working_data[column])
    plt.title(f"Boxplot of {column}")
    plt.show()

# 2.6.2 remove outliers (IQR)
columns = ["b","f","k","p","l","v","w"]
working_data = dp.remove_outliers(working_data,columns)
working_data.to_csv(r"../Data/cleaning_step_outliers.csv", index=False)

2.7 Imputing missing values

I used visualization from Data Wrangler to see the data distribution from each column. Based on distribution, I manually divided the colums into suitable methods: mean for normal distribution, median for skewed distribution and mode for discrete values.

Stats after:

Data shape      690 rows x 17 columns

Rows with missing values    0 (0.0%)

In [ ]:
# 2.7 impute missing values
mean_columns = ["b","f","k","l","p","r","v","w"]
median_columns = ["h","t"]
mode_columns = ["target","a","i"]
working_data = dp.impute_missing_values(working_data, mean_columns, median_columns, mode_columns)
working_data.to_csv(r"../Data/cleaning_step6.csv", index=False)

2.8 Normalize the data

I chose the MinMax method.

Stats after are same as the step before.

In [ ]:
# 2.8 normalize the data
numeric_columns = mean_columns+median_columns
clean_data = dp.normalize_data(working_data,numeric_columns)
clean_data.to_csv(r"../Data/clean_data.csv", index=False)

Step 3: Use the model

Use the model to calculate it's accuracy and compare messy data (0.8) to clean data (0.855). The accuracy increased by 5.5%

In [ ]:
### Train and evaluate the model
model_results = dp.simple_model(messy_data)
print(model_results)
model_results = dp.simple_model(clean_data)
print(model_results)